In [801]:
import os, math, numpy as np
from copy import deepcopy

In [802]:
def get_travel_time(source: tuple, destination : tuple) -> float:
    return math.dist(source, destination) / 427

In [803]:
### Calculate travel time between two points: origin and destination
def traveltime(origin_id,destination_id,meters_per_minute,locations):
    dist=np.sqrt((locations.at[destination_id,'x']-locations.at[origin_id,'x'])**2\
                +(locations.at[destination_id,'y']-locations.at[origin_id,'y'])**2)
    tt=np.ceil(dist/meters_per_minute)
    return tt

# Read in data instance
A folder of data instance has 5 files: (1) couriers.txt, (2) instance_characteristics.txt, (3) instance_parameters.txt, (4) orders.txt, (5) restaurants.txt

In [804]:
# Read instance information
def read_instance_information(instance_dir):
    orders=pd.read_table(os.path.join(instance_dir,'orders.txt'))
    restaurants=pd.read_table(os.path.join(instance_dir,'restaurants.txt'))
    couriers=pd.read_table(os.path.join(instance_dir,'couriers.txt'))
    instanceparams=pd.read_table(os.path.join(instance_dir,'instance_parameters.txt'))

    order_locations=pd.DataFrame(data=[orders.order,orders.x,orders.y]).transpose()
    order_locations.columns=['id','x','y']
    restaurant_locations=pd.DataFrame(data=[restaurants.restaurant,restaurants.x,restaurants.y]).transpose()
    restaurant_locations.columns=['id','x','y']
    courier_locations=pd.DataFrame(data=[couriers.courier,couriers.x,couriers.y]).transpose()
    courier_locations.columns=['id','x','y']
    locations=pd.concat([order_locations,restaurant_locations,courier_locations])
    # locations.set_index('id',inplace=True)
    #
    # orders.set_index('order',inplace=True)
    # couriers.set_index('courier',inplace=True)
    # restaurants.set_index('restaurant',inplace=True)

    meters_per_minute=instanceparams.at[0,'meters_per_minute']
    pickup_service_minutes=instanceparams.at[0,'pickup service minutes']
    dropoff_service_minutes=instanceparams.at[0,'dropoff service minutes']
    target_click_to_door=instanceparams.at[0,'target click-to-door']
    pay_per_order=instanceparams.at[0,'pay per order']
    guaranteed_pay_per_hour=instanceparams.at[0,'guaranteed pay per hour']

    return orders,restaurants,couriers,instanceparams,locations

In [805]:
import pandas as pd
instance_dir = 'data/0o50t75s1p100'
orders,restaurants,couriers,instanceparams,locations=read_instance_information(instance_dir)

# Set hyper-parameters:
f∶ every f minutes solves a matching problem to prescribe the next pick-up and delivery assignment for each courier<br>
t : optimization time<br>
∆_(u): t+ ∆_(u ) is the assignment horizon<br>
∆_(1); ∆_(2): to determine Z_(t)<br>
beta: control the freshness in the construction of bundles
gamma: control the click to door time in the construction of bundles


In [806]:
# Accorting to default values in the paper
f = 5
delta_u = 10
delta_1 = 10
delta_2 = 10
# Not find in the paper
beta = 10 # should be tuned
gamma = 10 # should be tuned

In [807]:
restaurant_dict = restaurants.set_index('restaurant').to_dict()

# Procedure to initialize a solution

**Function that return U(t,r)**<br>
U(t,r) is the set of upcoming orders at restaurant r

**Courier Object**

In [808]:
class Order:
    def __init__(self, order_information : dict):

        self.order_id = order_information.get('order')
        self.destination = (order_information.get('x'), order_information.get('y'))
        self.placement_time = order_information.get('placement_time')
        self.restaurant_id = order_information.get('restaurant')
        self.ready_time = order_information.get('ready_time')

        self.restaurant_coord = (restaurant_dict['x'][self.restaurant_id], restaurant_dict['y'][self.restaurant_id])
        self.delivered_time = None

    def __repr__(self):
        return "order_id : {} restaurant_location : {} destination : {} placement_time : {} restaurant_id : {} ready_time : {} delivered_time : ".format(
            self.order_id, self.restaurant_coord, self.destination, self.placement_time, self.restaurant_id, self.ready_time, self.delivered_time
        )


In [1009]:
class Courier(object):
    def __init__(self, courier_information : dict, restaurant : dict):
        self.courier = courier_information.get('courier')
        self.on_time = courier_information.get('on_time')
        self.off_time = courier_information.get('off_time')

        # Update and derived along the way
        self.visited = [(courier_information.get('x'), courier_information.get('y'))]
        self.time_location = [dict(time = self.on_time, location =  (courier_information.get('x'), courier_information.get('y')))]

    @staticmethod
    def calculate_travel_time(source: tuple,
                              restaurant:tuple,
                              destination: tuple) -> float:

        return get_travel_time(source, restaurant) + get_travel_time(restaurant, destination)

   # check if the courier can take the order
    def can_assign(self, order : Order) -> bool:
        # if the ready time < courier's off time
        if order.ready_time < self.on_time:
            return False
        if order.ready_time > self.off_time:
            return False

    # assign an order to courier
    def assign_order(self, order: Order) -> bool:
        # update available time for the next assignment
        _, restaurant_x, restaurant_y = restaurants[restaurants['restaurant'] == order.restaurant].values[0]
        if self.can_assign(order):
            self.orders.append(order)
            self.interim_time += self.calculate_travel_time(self.interim_position, (restaurant_x, restaurant_y), orders.destination)

    def __repr__(self):
        return "courier_id : {} current_coord : {} on_time : {} off_time : {} current_time : {}".format(
            self.courier, self.visited, self.on_time, self.off_time, self.time_location
        )

In [1161]:
class DeliveryRouting:
    def __init__(self, instance_dir : str, target_bundle_size : int = 5, f : int = 5, delta_u : int = 10):
        self.target_bundle_size = target_bundle_size
        self.delta_u = delta_u
        self.f = f

        orders, restaurants, couriers, instanceparams, locations = read_instance_information(instance_dir)

        self.orders = [Order(order) for order in orders.to_dict(orient = 'records')]
        self.orders = sorted(self.orders, key = lambda x: x.ready_time)

        self.restaurants = restaurants.to_dict(orient = 'records')
        self.couriers = [Courier(courier, self.restaurants) for courier in couriers.to_dict(orient = 'records')]
        self.unassigned_orders = deepcopy(self.orders)

        self.instance_params = instanceparams.to_dict(orient = 'records')[0]

        self.meters_per_minute, self.pickup_service_minutes, self.dropoff_service_minutes, \
            self.target_click_to_door, self.maximum_click_to_door, self.pay_per_order,\
            self.guaranteed_pay_per_hour = self.instance_params.values()

        self.bundles = {}
        self.bundles_assigned = {}


    @staticmethod
    def calculate_travel_time(source: tuple,
                              restaurant:tuple,
                              destination: tuple = None) -> float:

        if destination is not None:
            return get_travel_time(source, restaurant) + get_travel_time(restaurant, destination)
        return get_travel_time(source, restaurant)


    def get_bundle_size(self, z : int) -> int :
        # available = [courier for courier in self.couriers if courier.]
        pass

    def __is_unassigned(self,
                      order_check : Order) -> bool:

        for order in self.unassigned_orders:
            if order.order_id == order_check.order_id:
                return True
        return False

    def __assign_order(self,
                       courier : Courier,
                       to_assign_order : Order) -> None:

        for order in self.unassigned_orders:
            if order.order_id == to_assign_order.order_id:
                self.unassigned_orders.remove(order)

    def __get_available_couriers(self,
                                 t : int):
        return [courier for courier in self.couriers if courier.on_time <= t and courier.off_time >= t]

    def get_available_couriers(self,
                                 t : int):
        return [courier for courier in self.couriers if courier.on_time <= t and courier.off_time >= t]

    def __get_available_orders(self,
                               t : int):
        return [order for order in self.unassigned_orders if order.ready_time <= t + self.delta_u]

    def __get_courier(self, courier_id : str):
        for courier in self.couriers:
            if courier.courier == courier_id:
                return courier

    def check_can_bundle(self,
                         courier : Courier,
                         existing_bundle : [Order],
                         new_order : Order,
                         is_update = False ):

        travel_time = self.calculate_travel_time(courier.visited[-1], existing_bundle[-1].restaurant_coord)
        max_leave_time = max([order.ready_time for order in existing_bundle])
        max_leave_time = max(travel_time, max_leave_time)
        max_leave_time = max(max_leave_time, new_order.ready_time) + 4

        if max_leave_time >= existing_bundle[0].ready_time + 90:
            return False

        for order in existing_bundle:
            if order.restaurant_id != new_order.restaurant_id:
                return False

        if new_order.ready_time > courier.off_time:
            return False

        if new_order.ready_time < courier.on_time:
            return False


        start_position = courier.visited.index(existing_bundle[0].restaurant_coord) + 1
        time_location = deepcopy(courier.time_location[:start_position])
        visited = deepcopy(courier.visited[:start_position])

        for i in range(len(existing_bundle)-1):
            if i == 0:
                time_location.append(dict(time = max_leave_time, location =  existing_bundle[i].destination))
                visited.append(existing_bundle[i].destination)

            max_leave_time += self.calculate_travel_time(existing_bundle[i].destination, existing_bundle[i+1].destination) + 4
            time_location.append(dict(time = max_leave_time, location =  existing_bundle[i+1].destination))
            visited.append(existing_bundle[i+1].destination)

        if is_update == True:
            courier.visited = visited
            courier.time_location = time_location
        return max_leave_time

    def assign_task(self, x : int):

        available_order = self.__get_available_orders(x)
        available_courier = self.__get_available_couriers(x)

        for order in available_order:
            for bundle_id, bundle_list in self.bundles.items():
                if self.__is_unassigned(order):
                    courier_assigned = self.bundles_assigned[bundle_id]

                    if self.check_can_bundle(courier_assigned, bundle_list, order):
                        bundle_list.append(order)
                        self.check_can_bundle(courier_assigned, bundle_list, order, True)
                        self.bundles[bundle_id] = bundle_list
                        self.__assign_order(courier_assigned, order)

            if self.__is_unassigned(order):
                min_time = 1e9
                is_assigned = False
                for courier in available_courier:
                    time_taken = self.calculate_travel_time(courier.visited[-1], order.restaurant_coord, order.destination)
                    if time_taken < min_time and courier.time_location[-1]['time'] + time_taken < courier.off_time:
                        min_time = time_taken
                        courier_assigned = courier
                        is_assigned = True


                if is_assigned:
                    time_travelled = max([task['time'] for task in courier_assigned.time_location])

                    if len(courier_assigned.time_location) > 1:
                        time_travelled += 4

                    courier_assigned.time_location.append(dict(time = time_travelled + self.calculate_travel_time(courier.visited[-1], order.restaurant_coord), location = order.restaurant_coord))

                    time_travelled = max([task['time'] for task in courier_assigned.time_location])
                    to_leave = max(time_travelled, order.ready_time) + 4

                    courier_assigned.time_location.append(dict(time = time_travelled + self.calculate_travel_time(order.restaurant_coord, order.destination), location = order.destination))

                    courier_assigned.visited.extend([order.restaurant_coord, order.destination])

                    key = len(self.bundles)
                    self.bundles[key] = [order]
                    self.bundles_assigned[key] = courier_assigned
                    self.__assign_order(courier_assigned, order)

In [1166]:
instance_dir = 'data/0o50t75s1p100'
dr = DeliveryRouting(instance_dir)
dr.assign_task(180)

In [1169]:
dr.bundles

{0: [order_id : o146 restaurant_location : (7856, 5960) destination : (3695, 3690) placement_time : 13 restaurant_id : r54 ready_time : 28 delivered_time : ],
 1: [order_id : o89 restaurant_location : (6638, 2098) destination : (7663, 2842) placement_time : 24 restaurant_id : r50 ready_time : 29 delivered_time : ],
 2: [order_id : o240 restaurant_location : (9472, 3738) destination : (9386, 5834) placement_time : 29 restaurant_id : r67 ready_time : 46 delivered_time : ],
 3: [order_id : o227 restaurant_location : (4847, 5399) destination : (9334, 6307) placement_time : 33 restaurant_id : r39 ready_time : 48 delivered_time : ,
  order_id : o163 restaurant_location : (4847, 5399) destination : (4058, 2280) placement_time : 72 restaurant_id : r39 ready_time : 95 delivered_time : ],
 4: [order_id : o159 restaurant_location : (9472, 3738) destination : (10213, 5982) placement_time : 36 restaurant_id : r67 ready_time : 56 delivered_time : ],
 5: [order_id : o121 restaurant_location : (3859, 

In [1170]:
dr.unassigned_orders

[order_id : o152 restaurant_location : (2213, 5564) destination : (342, 2729) placement_time : 177 restaurant_id : r11 ready_time : 197 delivered_time : ,
 order_id : o218 restaurant_location : (7644, 3559) destination : (9470, 4296) placement_time : 184 restaurant_id : r68 ready_time : 199 delivered_time : ,
 order_id : o114 restaurant_location : (7970, 5491) destination : (9679, 5989) placement_time : 171 restaurant_id : r56 ready_time : 201 delivered_time : ,
 order_id : o119 restaurant_location : (8003, 4381) destination : (8076, 8206) placement_time : 187 restaurant_id : r18 ready_time : 202 delivered_time : ,
 order_id : o235 restaurant_location : (8694, 5518) destination : (9093, 4164) placement_time : 201 restaurant_id : r91 ready_time : 202 delivered_time : ,
 order_id : o66 restaurant_location : (6899, 3160) destination : (3820, 5225) placement_time : 198 restaurant_id : r5 ready_time : 203 delivered_time : ,
 order_id : o27 restaurant_location : (4018, 5802) destination : (8

**Funtion that calculate bundel size Z(t)**

**Function that calculate k(r,t)**<br>
k(r,t) is the number of couriers who are available at restaurant r

In [545]:
def get_available_courier(t,r):
    ...

**Route object**<br>
A route is a bundle,i.e: a list of ordered orders

In [ ]:
class Route(object):
    def __init__:
    
    # calculate total travel time from 1st destination to the last destination of the route
    def total_travel_time():

    # calculate total service delay    
    def total_service_delay():
        #<to be discuss>

    # calculate total_click_to_door 
    def total_click_to_door():
        #<to be discuss>

    # calculate route efficiency: travel time per order:
    def route_efficiency():

    # calculate route cost
    def route_cost():
        #route_cost = total_travel_time + beta*total_service_delay + gamma*total_click_to_door


# Implement procedure 1